In [1]:
# Notebook 02 - TRANSFORMAÇÃO DE DADOS

# ETAPA 1: CARREGAR DADOS DA CAMADA BRONZE

import pandas as pd
import numpy as np
import os
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

print("Verificando estrutura de pastas...")

pastas = [
    '../dados/bronze',
    '../dados/silver',
    '../dados/gold'
]

for pasta in pastas:
    os.makedirs(pasta, exist_ok=True)

print("Estrutura de pastas verificada!")
print("   - /dados/bronze")
print("   - /dados/silver")
print("   - /dados/gold\n")

print("="*60)
print("INICIANDO TRANSFORMAÇÃO DE DADOS")
print("="*60)

df_silver = pd.read_csv('../dados/bronze/telco_churn_raw.csv')
print(f"\nDados carregados da camada Bronze: {len(df_silver):,} registros")

df_original_shape = df_silver.shape


Verificando estrutura de pastas...
Estrutura de pastas verificada!
   - /dados/bronze
   - /dados/silver
   - /dados/gold

INICIANDO TRANSFORMAÇÃO DE DADOS

Dados carregados da camada Bronze: 7,043 registros


In [2]:
# ETAPA 2: LIMPEZA DE DADOS

print("\n" + "="*60)
print("LIMPEZA DE DADOS")
print("="*60)

print("\nConvertendo TotalCharges para numérico...")

print(f"Tipo antes: {df_silver['TotalCharges'].dtype}")

df_silver['TotalCharges'] = pd.to_numeric(df_silver['TotalCharges'], errors='coerce')

valores_vazios = df_silver['TotalCharges'].isnull().sum()
print(f"   Valores vazios encontrados: {valores_vazios}")
print(f"   Tipo depois: {df_silver['TotalCharges'].dtype}")

print("\nTratando valores ausentes...")

df_silver.loc[df_silver['tenure'] == 0, 'TotalCharges'] = 0

mask = (df_silver['TotalCharges'].isnull()) & (df_silver['tenure'] > 0)
df_silver.loc[mask, 'TotalCharges'] = df_silver.loc[mask, 'MonthlyCharges'] * df_silver.loc[mask, 'tenure']

print(f"   Valores preenchidos: {valores_vazios}")

duplicatas_antes = df_silver.duplicated(subset='customerID').sum()
df_silver = df_silver.drop_duplicates(subset='customerID', keep='first')
print(f"\n🔧 Duplicatas removidas: {duplicatas_antes}")

ausentes_final = df_silver.isnull().sum().sum()
print(f"\nTotal de valores ausentes após limpeza: {ausentes_final}")

print("\n🔧 Padronizando valores categóricos...")
df_silver['Churn_Binary'] = (df_silver['Churn'] == 'Yes').astype(int)
print("   - Churn convertido para binário (0/1)")

print("\n" + "="*60)
print("LIMPEZA COMPLETA!")
print("="*60)




LIMPEZA DE DADOS

Convertendo TotalCharges para numérico...
Tipo antes: object
   Valores vazios encontrados: 11
   Tipo depois: float64

Tratando valores ausentes...
   Valores preenchidos: 11

🔧 Duplicatas removidas: 0

Total de valores ausentes após limpeza: 0

🔧 Padronizando valores categóricos...
   - Churn convertido para binário (0/1)

LIMPEZA COMPLETA!


In [3]:
# ETAPA 3: ENGENHARIA DE FEATURES

print("\n" + "="*60)
print("ENGENHARIA DE FEATURES")
print("="*60)

df_silver['AvgChargePerMonth'] = df_silver['TotalCharges'] / (df_silver['tenure'] + 1)
print("\nFeature criada: AvgChargePerMonth")

def categorizar_tenure(tenure):
    if tenure <= 12:
        return 'Novo'
    elif tenure <= 36:
        return 'Medio'
    else:
        return 'Longo'

df_silver['TenureGroup'] = df_silver['tenure'].apply(categorizar_tenure)
print("Feature criada: TenureGroup (Novo/Medio/Longo)")

print("\n   Distribuição de TenureGroup:")
print(df_silver['TenureGroup'].value_counts())

mediana_charges = df_silver['MonthlyCharges'].median()
df_silver['IsPremium'] = (df_silver['MonthlyCharges'] > mediana_charges).astype(int)
print(f"\nFeature criada: IsPremium (threshold: R$ {mediana_charges:.2f})")
print(f"   - Clientes Premium: {df_silver['IsPremium'].sum():,}")
print(f"   - Clientes Regular: {(df_silver['IsPremium']==0).sum():,}")

servicos = ['PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity',
            'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']

def contar_servicos(row):
    """
    Conta o número de serviços ativos contratados pelo cliente
    """
    count = 0
    valores_negativos = ['No', 'No phone service', 'No internet service']

    for servico in servicos:
        if servico in row.index:
            valor = row[servico]
            # Verificar se é string e não está na lista de valores negativos
            if isinstance(valor, str) and valor not in valores_negativos:
                count += 1
            # Verificar se é Yes (caso seja diferente de string por algum motivo)
            elif valor == 'Yes':
                count += 1

    return count

df_silver['NumServicos'] = df_silver.apply(contar_servicos, axis=1)
print("\nFeature criada: NumServicos (total de serviços contratados)")
print(f"   - Média de serviços: {df_silver['NumServicos'].mean():.2f}")
print(f"   - Min: {df_silver['NumServicos'].min()} | Max: {df_silver['NumServicos'].max()}")

print("\nDistribuição de serviços contratados:")
print(df_silver['NumServicos'].value_counts().sort_index())

df_silver['HasSecurity'] = ((df_silver['OnlineSecurity'] == 'Yes') |
                             (df_silver['DeviceProtection'] == 'Yes')).astype(int)
print(f"\nFeature criada: HasSecurity")
print(f"   - Com segurança: {df_silver['HasSecurity'].sum():,}")
print(f"   - Sem segurança: {(df_silver['HasSecurity']==0).sum():,}")

df_silver['SatisfactionScore'] = (
    (df_silver['Contract'] != 'Month-to-month').astype(int) * 3 +
    (df_silver['NumServicos'] / df_silver['NumServicos'].max()) * 3 +
    (df_silver['TechSupport'] == 'Yes').astype(int) * 2 +
    (df_silver['tenure'] / df_silver['tenure'].max()) * 2
)
print("\nFeature criada: SatisfactionScore (0-10)")
print(f"   - Score médio: {df_silver['SatisfactionScore'].mean():.2f}")
print(f"   - Score mínimo: {df_silver['SatisfactionScore'].min():.2f}")
print(f"   - Score máximo: {df_silver['SatisfactionScore'].max():.2f}")

df_silver['ChurnRiskScore'] = (
    (df_silver['Contract'] == 'Month-to-month').astype(int) * 3 +
    (df_silver['tenure'] < 12).astype(int) * 2 +
    (df_silver['NumServicos'] <= 2).astype(int) * 2 +
    (df_silver['HasSecurity'] == 0).astype(int) * 1 +
    (df_silver['MonthlyCharges'] > mediana_charges).astype(int) * 1
)
print("\nFeature criada: ChurnRiskScore (0-9)")
print(f"   - Score médio: {df_silver['ChurnRiskScore'].mean():.2f}")



ENGENHARIA DE FEATURES

Feature criada: AvgChargePerMonth
Feature criada: TenureGroup (Novo/Medio/Longo)

   Distribuição de TenureGroup:
TenureGroup
Longo    3001
Novo     2186
Medio    1856
Name: count, dtype: int64

Feature criada: IsPremium (threshold: R$ 70.35)
   - Clientes Premium: 3,515
   - Clientes Regular: 3,528

Feature criada: NumServicos (total de serviços contratados)
   - Média de serviços: 4.15
   - Min: 1 | Max: 9

Distribuição de serviços contratados:
NumServicos
1    1264
2     859
3     846
4     965
5     922
6     908
7     676
8     395
9     208
Name: count, dtype: int64

Feature criada: HasSecurity
   - Com segurança: 3,330
   - Sem segurança: 3,713

Feature criada: SatisfactionScore (0-10)
   - Score médio: 4.21
   - Score mínimo: 0.36
   - Score máximo: 10.00

Feature criada: ChurnRiskScore (0-9)
   - Score médio: 3.87


In [4]:
# ETAPA 4: NORMALIZAÇÃO E AGREGAÇÕES

print("\n" + "="*60)
print("NORMALIZAÇÃO E AGREGAÇÕES")
print("="*60)

scaler = MinMaxScaler()
df_silver['MonthlyCharges_Normalized'] = scaler.fit_transform(df_silver[['MonthlyCharges']])
df_silver['TotalCharges_Normalized'] = scaler.fit_transform(df_silver[['TotalCharges']])
print("Colunas normalizadas: MonthlyCharges e TotalCharges (escala 0-1)")

print("\nCriando agregações estatísticas...")

contract_stats = df_silver.groupby('Contract').agg({
    'MonthlyCharges': ['mean', 'median', 'std'],
    'TotalCharges': ['mean', 'median'],
    'Churn_Binary': ['sum', 'mean']
}).round(2)

print("\nMédia por tipo de contrato:")
print(contract_stats)

tenure_stats = df_silver.groupby('TenureGroup').agg({
    'MonthlyCharges': 'mean',
    'NumServicos': 'mean',
    'Churn_Binary': 'mean'
}).round(2)

print("\n\nMédia por grupo de tenure:")
print(tenure_stats)



NORMALIZAÇÃO E AGREGAÇÕES
Colunas normalizadas: MonthlyCharges e TotalCharges (escala 0-1)

Criando agregações estatísticas...

Média por tipo de contrato:
               MonthlyCharges               TotalCharges           \
                         mean median    std         mean   median   
Contract                                                            
Month-to-month          66.40  73.25  26.93      1369.25   679.55   
One year                65.05  68.75  31.84      3032.62  2656.70   
Two year                60.77  64.35  34.68      3706.93  3593.80   

               Churn_Binary        
                        sum  mean  
Contract                           
Month-to-month         1655  0.43  
One year                166  0.11  
Two year                 48  0.03  


Média por grupo de tenure:
             MonthlyCharges  NumServicos  Churn_Binary
TenureGroup                                           
Longo                 72.01         5.22          0.12
Medio             

In [5]:
# ETAPA 5: VALIDAÇÃO DA QUALIDADE

print("\n" + "="*60)
print("✔️ VALIDAÇÃO DE QUALIDADE PÓS-TRANSFORMAÇÃO")
print("="*60)

print(f"\nTotal de registros após transformação: {len(df_silver):,}")
print(f"Colunas antes: {df_original_shape[1]}")
print(f"Colunas depois: {df_silver.shape[1]}")
print(f"Novas features criadas: {df_silver.shape[1] - df_original_shape[1]}")

print("\nVerificando valores ausentes:")
missing = df_silver.isnull().sum()
if missing.sum() > 0:
    print("Colunas com valores ausentes:")
    print(missing[missing > 0])
else:
    print("Nenhum valor ausente!")

print("\nRange das novas features:")
print(f"   - NumServicos: {df_silver['NumServicos'].min()} a {df_silver['NumServicos'].max()}")
print(f"   - SatisfactionScore: {df_silver['SatisfactionScore'].min():.2f} a {df_silver['SatisfactionScore'].max():.2f}")
print(f"   - ChurnRiskScore: {df_silver['ChurnRiskScore'].min():.2f} a {df_silver['ChurnRiskScore'].max():.2f}")
print(f"   - AvgChargePerMonth: R$ {df_silver['AvgChargePerMonth'].min():.2f} a R$ {df_silver['AvgChargePerMonth'].max():.2f}")

print("\nTipos de dados das novas features:")
novas_features = ['Churn_Binary', 'AvgChargePerMonth', 'TenureGroup', 'IsPremium',
                  'NumServicos', 'HasSecurity', 'SatisfactionScore', 'ChurnRiskScore',
                  'MonthlyCharges_Normalized', 'TotalCharges_Normalized']
for feat in novas_features:
    print(f"   - {feat}: {df_silver[feat].dtype}")



✔️ VALIDAÇÃO DE QUALIDADE PÓS-TRANSFORMAÇÃO

Total de registros após transformação: 7,043
Colunas antes: 23
Colunas depois: 33
Novas features criadas: 10

Verificando valores ausentes:
Nenhum valor ausente!

Range das novas features:
   - NumServicos: 1 a 9
   - SatisfactionScore: 0.36 a 10.00
   - ChurnRiskScore: 0.00 a 9.00
   - AvgChargePerMonth: R$ 0.00 a R$ 118.97

Tipos de dados das novas features:
   - Churn_Binary: int64
   - AvgChargePerMonth: float64
   - TenureGroup: object
   - IsPremium: int64
   - NumServicos: int64
   - HasSecurity: int64
   - SatisfactionScore: float64
   - ChurnRiskScore: int64
   - MonthlyCharges_Normalized: float64
   - TotalCharges_Normalized: float64


In [6]:
# ETAPA 6: SALVAR DADOS TRANSFORMADOS (CAMADA SILVER)

print("\n" + "="*60)
print("SALVANDO DADOS TRANSFORMADOS")
print("="*60)

caminho_silver_csv = '../dados/silver/telco_churn_transformed.csv'
df_silver.to_csv(caminho_silver_csv, index=False)
print(f"CSV salvo em: {caminho_silver_csv}")

caminho_silver_parquet = '../dados/silver/telco_churn_transformed.parquet'
df_silver.to_parquet(caminho_silver_parquet, index=False, engine='pyarrow')
print(f"Parquet salvo em: {caminho_silver_parquet}")

import os
tamanho_csv = os.path.getsize(caminho_silver_csv) / 1024 / 1024
tamanho_parquet = os.path.getsize(caminho_silver_parquet) / 1024 / 1024
reducao = ((tamanho_csv - tamanho_parquet) / tamanho_csv) * 100

print(f"\nComparação de formatos:")
print(f"   - CSV: {tamanho_csv:.2f} MB")
print(f"   - Parquet: {tamanho_parquet:.2f} MB")
print(f"   - Redução: {reducao:.1f}%")



SALVANDO DADOS TRANSFORMADOS
CSV salvo em: ../dados/silver/telco_churn_transformed.csv
Parquet salvo em: ../dados/silver/telco_churn_transformed.parquet

Comparação de formatos:
   - CSV: 1.77 MB
   - Parquet: 0.34 MB
   - Redução: 80.7%


In [7]:
# ETAPA 7: CRIAR DATASETS AGREGADOS (CAMADA GOLD)

print("\n" + "="*60)
print("CRIANDO DATASETS GOLD (PRONTOS PARA ANÁLISE)")
print("="*60)

df_gold_contract = df_silver.groupby('Contract').agg({
    'customerID': 'count',
    'Churn_Binary': ['sum', 'mean'],
    'MonthlyCharges': ['mean', 'median'],
    'TotalCharges': ['mean', 'median'],
    'tenure': ['mean', 'median'],
    'NumServicos': 'mean',
    'SatisfactionScore': 'mean'
}).round(2)

df_gold_contract.columns = ['_'.join(col).strip() for col in df_gold_contract.columns.values]
df_gold_contract.reset_index(inplace=True)
df_gold_contract.to_csv('../dados/gold/metricas_por_contrato.csv', index=False)
print("Dataset Gold criado: metricas_por_contrato.csv")
print(df_gold_contract)

df_gold_segments = df_silver.groupby(['TenureGroup', 'IsPremium']).agg({
    'customerID': 'count',
    'Churn_Binary': ['sum', 'mean'],
    'SatisfactionScore': 'mean',
    'NumServicos': 'mean'
}).round(2)

df_gold_segments.columns = ['_'.join(col).strip() for col in df_gold_segments.columns.values]
df_gold_segments.reset_index(inplace=True)
df_gold_segments.to_csv('../dados/gold/churn_por_segmento.csv', index=False)
print("\nDataset Gold criado: churn_por_segmento.csv")
print(df_gold_segments)

df_alto_risco = df_silver[df_silver['Churn_Binary'] == 1].copy()
df_alto_risco_summary = df_alto_risco.describe().T
df_alto_risco_summary.to_csv('../dados/gold/perfil_alto_risco.csv')
print("\nDataset Gold criado: perfil_alto_risco.csv")
print("\nEstatísticas dos clientes que fizeram churn:")
print(df_alto_risco[['tenure', 'MonthlyCharges', 'TotalCharges', 'NumServicos', 'SatisfactionScore']].describe())

numeric_cols = ['tenure', 'MonthlyCharges', 'TotalCharges', 'NumServicos',
                'SatisfactionScore', 'IsPremium', 'HasSecurity', 'ChurnRiskScore']
correlations = df_silver[numeric_cols + ['Churn_Binary']].corr()['Churn_Binary'].sort_values(ascending=False)
correlations_df = pd.DataFrame({
    'Feature': correlations.index,
    'Correlation': correlations.values
})
correlations_df.to_csv('../dados/gold/correlacoes_churn.csv', index=False)
print("\nDataset Gold criado: correlacoes_churn.csv")
print("\nTop correlações com Churn:")
print(correlations_df.head(10))

print("\n" + "="*60)
print("TRANSFORMAÇÃO COMPLETA!")
print("="*60)

relatorio = f"""
RELATÓRIO DE TRANSFORMAÇÃO DE DADOS
Data: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
==========================================
Registros processados: {len(df_silver):,}
Features originais: {df_original_shape[1]}
Features após engenharia: {df_silver.shape[1]}
Novas features criadas: {df_silver.shape[1] - df_original_shape[1]}
Datasets Gold criados: 4
==========================================

NOVAS FEATURES CRIADAS:
1. Churn_Binary - Conversão binária da variável alvo
2. AvgChargePerMonth - Valor médio por mês de contrato
3. TenureGroup - Categorização de tempo (Novo/Médio/Longo)
4. IsPremium - Flag de cliente premium (acima da mediana)
5. NumServicos - Total de serviços contratados
6. HasSecurity - Flag de serviços de segurança
7. SatisfactionScore - Score estimado de satisfação (0-10)
8. ChurnRiskScore - Score de risco de churn (0-9)
9. MonthlyCharges_Normalized - Normalização 0-1
10. TotalCharges_Normalized - Normalização 0-1

==========================================

QUALIDADE DOS DADOS:
- Valores ausentes: {df_silver.isnull().sum().sum()}
- Duplicatas: 0
- Tipos de dados: Validados
- Ranges: Consistentes

==========================================

DATASETS GOLD GERADOS:
1. metricas_por_contrato.csv - KPIs por tipo de contrato
2. churn_por_segmento.csv - Análise de segmentos
3. perfil_alto_risco.csv - Características de clientes em risco
4. correlacoes_churn.csv - Features mais correlacionadas

==========================================

OTIMIZAÇÕES APLICADAS:
- Formato Parquet: Redução de {reducao:.1f}% no tamanho
- Normalização: Escalas 0-1 para ML futuro
- Agregações: Pré-computadas para análise rápida

==========================================
"""

with open('../dados/silver/relatorio_transformacao.txt', 'w', encoding='utf-8') as f:
    f.write(relatorio)

print("\nRelatório salvo em: ../dados/silver/relatorio_transformacao.txt")

print("\n" + "="*60)
print("RESUMO FINAL DA TRANSFORMAÇÃO")
print("="*60)
print(f"\nPipeline de transformação executado com sucesso!")
print(f"{len(df_silver):,} registros processados")
print(f"{df_silver.shape[1] - df_original_shape[1]} novas features criadas")
print(f"4 datasets Gold gerados")
print(f"Qualidade dos dados: 100% validada")
print(f"Dados prontos para análise e visualização!")
print("\nPróximo passo: Execute o notebook 03_analise_visualizacao.ipynb")
print("="*60)


CRIANDO DATASETS GOLD (PRONTOS PARA ANÁLISE)
Dataset Gold criado: metricas_por_contrato.csv
         Contract  customerID_count  Churn_Binary_sum  Churn_Binary_mean  \
0  Month-to-month              3875              1655               0.43   
1        One year              1473               166               0.11   
2        Two year              1695                48               0.03   

   MonthlyCharges_mean  MonthlyCharges_median  TotalCharges_mean  \
0                66.40                  73.25            1369.25   
1                65.05                  68.75            3032.62   
2                60.77                  64.35            3706.93   

   TotalCharges_median  tenure_mean  tenure_median  NumServicos_mean  \
0               679.55        18.04           12.0              3.70   
1              2656.70        42.04           44.0              4.58   
2              3593.80        56.74           64.0              4.79   

   SatisfactionScore_mean  
0           